## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,-53.1500,-70.9167,41.02,87,0,4.61,clear sky
1,1,Bethel,US,41.3712,-73.4140,50.68,90,100,11.50,overcast clouds
2,2,Vaini,TO,-21.2000,-175.2000,77.16,100,75,3.44,light intensity shower rain
3,3,Busselton,AU,-33.6500,115.3333,65.44,59,65,4.09,broken clouds
4,4,Pevek,RU,69.7008,170.3133,-3.30,98,100,8.52,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
#Prompt the customer for the minimum  and maximum temperature preference, rain and snow.
print("What is your minimum temperature preference?")
min_temp = float(input("min_temp: "))

print("What is your maximum temperature preference?")
max_temp = float(input("max_temp: "))


What is your minimum temperature preference?
min_temp: 75
What is your maximum temperature preference?
max_temp: 88


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preffered_cities_df = city_data_df  .loc[(city_data_df ["Max Temp"] > min_temp) & (city_data_df ["Max Temp"] < max_temp)]
preffered_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Vaini,TO,-21.2000,-175.2000,77.16,100,75,3.44,light intensity shower rain
6,6,Avarua,CK,-21.2078,-159.7750,80.65,83,40,5.75,scattered clouds
7,7,Liku,ID,-0.7275,119.8922,75.34,90,100,1.79,overcast clouds
28,28,Imbituba,BR,-28.2400,-48.6703,81.82,63,69,11.45,broken clouds
30,30,Tautira,PF,-17.7333,-149.1500,80.98,78,21,17.16,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preffered_cities_df.count()

City_ID                216
City                   216
Country                214
Lat                    216
Lng                    216
Max Temp               216
Humidity               216
Cloudiness             216
Wind Speed             216
Current Description    216
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_data_df = preffered_cities_df.dropna()
clean_data_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Vaini,TO,-21.2000,-175.2000,77.16,100,75,3.44,light intensity shower rain
6,6,Avarua,CK,-21.2078,-159.7750,80.65,83,40,5.75,scattered clouds
7,7,Liku,ID,-0.7275,119.8922,75.34,90,100,1.79,overcast clouds
28,28,Imbituba,BR,-28.2400,-48.6703,81.82,63,69,11.45,broken clouds
30,30,Tautira,PF,-17.7333,-149.1500,80.98,78,21,17.16,light rain
...,...,...,...,...,...,...,...,...,...,...
662,662,Caravelas,BR,-17.7125,-39.2481,81.19,78,48,4.88,scattered clouds
663,663,Buala,SB,-8.1450,159.5921,80.11,82,100,6.04,light rain
669,669,Oistins,BB,13.0667,-59.5333,86.09,58,20,19.57,few clouds
670,670,Makokou,GA,0.5738,12.8642,84.09,61,93,1.12,overcast clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_data_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Vaini,TO,77.16,light intensity shower rain,-21.2000,-175.2000,
6,Avarua,CK,80.65,scattered clouds,-21.2078,-159.7750,
7,Liku,ID,75.34,overcast clouds,-0.7275,119.8922,
28,Imbituba,BR,81.82,broken clouds,-28.2400,-48.6703,
30,Tautira,PF,80.98,light rain,-17.7333,-149.1500,
36,Casian,PH,80.06,overcast clouds,11.0350,119.7136,
39,Kudahuvadhoo,MV,84.07,broken clouds,2.6708,72.8944,
42,Bandar-E Lengeh,IR,86.11,clear sky,26.5579,54.8807,
43,Hithadhoo,MV,82.90,clear sky,-0.6000,73.0833,
45,Arraial Do Cabo,BR,80.56,few clouds,-22.9661,-42.0278,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Cannot find hotel...skipping.")  

Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
c = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = c.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Vaini,TO,77.16,light intensity shower rain,-21.2000,-175.2000,Keleti Beach Resort
6,Avarua,CK,80.65,scattered clouds,-21.2078,-159.7750,Paradise Inn
7,Liku,ID,75.34,overcast clouds,-0.7275,119.8922,Wahyu
28,Imbituba,BR,81.82,broken clouds,-28.2400,-48.6703,Praia Hotel Imbituba
30,Tautira,PF,80.98,light rain,-17.7333,-149.1500,Pension Oaoa
...,...,...,...,...,...,...,...
662,Caravelas,BR,81.19,scattered clouds,-17.7125,-39.2481,Pousada dos Navegantes
663,Buala,SB,80.11,light rain,-8.1450,159.5921,Maringe Lagoon Lodge
669,Oistins,BB,86.09,few clouds,13.0667,-59.5333,Butterfly Beach Hotel
670,Makokou,GA,84.09,overcast clouds,0.5738,12.8642,Hôtel


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_search/weatherpy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dt><dd>{Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))